In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
ds = load_dataset("stdt1/nfcc") # Loading dataset
print(ds)

DatasetDict({
    train: Dataset({
        features: ['definition', 'type'],
        num_rows: 5100
    })
    test: Dataset({
        features: ['definition', 'type'],
        num_rows: 900
    })
})


In [5]:
train_ds = ds["train"]
test_ds = ds["test"]

print(train_ds[5])
print(test_ds[2])

{'definition': 'Réole [ reå11 ], La, arrondissemangshufvudstad i franska dep. Gironde, på högra stranden af Ga - ronne, vid sydbanan. 4, 241 inv. ( 1911 ) som kommun. College. Fabrikation af brännvin och likör. ( J - F. N. )', 'type': 1}
{'definition': 'Trollhätte kraftverk, Sveriges till storleken tredje kraftverk ( 1954, efter Harsprånget och Kilforsen ), utnyttjar vattenkraften i Trollhättefallen med sammanlagt 253, 000 kW installerad turbineffekt vid c : a 31 m nettofallhöjd ( uttagbar effekt c : a 225, 000 kW ). Till kraftförvaltningen T. höra även Vargöns kraftverk och Lilla Edets kraftverk. — T. var det första av statens kraftverk, tillkommet genom beslut av 1906 års riksdag', 'type': 0}


In [6]:
model_name = "KB/bert-base-swedish-cased" # KB-BERT
num_classes = 3 # other, location, person
max_length = 128
batch_size = 16
num_epochs = 4
learning_rate = 2e-5

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [8]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
        return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'label': torch.tensor(label)}

In [9]:
tok_train_ds = TextClassificationDataset(train_ds["definition"], train_ds["type"], tokenizer, max_length)
tok_test_ds = TextClassificationDataset(test_ds["definition"], test_ds["type"], tokenizer, max_length)

train_dl = DataLoader(tok_train_ds, batch_size=batch_size, shuffle=True)
test_dl = DataLoader(tok_test_ds, batch_size=batch_size)

In [10]:
class EntityClassifier(nn.Module):
    def __init__(self, model_name, num_classes):
        super(EntityClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits

In [11]:
model = EntityClassifier(model_name, num_classes).to(device)
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dl) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

In [12]:
def train(model, data_loader, optimizer, scheduler, device):
    model.train()
    for batch in tqdm(data_loader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

In [13]:
def evaluate(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())
    return accuracy_score(actual_labels, predictions), classification_report(actual_labels, predictions)

In [14]:
def num_to_class(num):
    if num == 0:
        return "other"
    elif num == 1:
        return "location"
    elif num == 2:
        return "person"
    else:
        return "something went wrong"

In [15]:
def predict_sentiment(text, model, tokenizer, device, max_length=128):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)
    return num_to_class(preds.item())

In [16]:
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train(model, train_dl, optimizer, scheduler, device)
    accuracy, report = evaluate(model, test_dl, device)
    print(f"Validation Accuracy: {accuracy:.4f}")
    print(report)

Epoch 1/4


100%|██████████| 319/319 [02:44<00:00,  1.94it/s]


Validation Accuracy: 0.9333
              precision    recall  f1-score   support

           0       0.95      0.87      0.91       330
           1       0.92      0.96      0.94       330
           2       0.93      0.99      0.96       240

    accuracy                           0.93       900
   macro avg       0.93      0.94      0.93       900
weighted avg       0.93      0.93      0.93       900

Epoch 2/4


100%|██████████| 319/319 [02:43<00:00,  1.95it/s]


Validation Accuracy: 0.9422
              precision    recall  f1-score   support

           0       0.95      0.89      0.92       330
           1       0.91      0.98      0.94       330
           2       0.98      0.97      0.97       240

    accuracy                           0.94       900
   macro avg       0.95      0.94      0.94       900
weighted avg       0.94      0.94      0.94       900

Epoch 3/4


100%|██████████| 319/319 [02:43<00:00,  1.95it/s]


Validation Accuracy: 0.9367
              precision    recall  f1-score   support

           0       0.92      0.91      0.91       330
           1       0.93      0.95      0.94       330
           2       0.97      0.95      0.96       240

    accuracy                           0.94       900
   macro avg       0.94      0.94      0.94       900
weighted avg       0.94      0.94      0.94       900

Epoch 4/4


100%|██████████| 319/319 [02:43<00:00,  1.95it/s]


Validation Accuracy: 0.9356
              precision    recall  f1-score   support

           0       0.94      0.89      0.91       330
           1       0.92      0.96      0.94       330
           2       0.95      0.96      0.96       240

    accuracy                           0.94       900
   macro avg       0.94      0.94      0.94       900
weighted avg       0.94      0.94      0.94       900



In [65]:
torch.save(model.state_dict(), "best_entity_classifier_so_far.pth")

In [17]:
# Manual test
test_string = "Karl Magnus, sjöofficer, statsråd, f. 4 mars 1803 i Karlshamn, d. 18 jan. 1874 på Agdatorp, Nättraby socken, Blekinge. Efter att 1821 hafva blifvit underlöjtnant vid örlogsflottan tjänstgjorde han till 1825 på amerikanska handelsfartyg för att utbilda sig i sitt yrke. 1824 utnämnd till sekundlöjtnant, anställdes han 1828 vid franska flottan, med hvilken han deltog i blockaden af Lissabon (1831). 1830 befordrades han till premiärlöjtnant vid svenska flottan och fick såsom sådan befäl på örlogsångaren Odin, med hvilken han 1836 stötte på grund vid Jyllands kust. Denna händelse väckte på sin tid stort uppseende och utsatte E. för mycket tadel; men sedermera tror man sig hafva funnit, att olyckan härrörde af en tillfällighet, för hvilken chefen icke var ansvarig. 1846 blef E. kapten, 1847 chef för Sjöförsvarsdep:s kommandoexpedition och 1853 kommendörkapten. 1857 utnämndes han till statsråd och chef för Sjöförsvarsdepartementet. När han 1862 nedlade detta ämbete, blef han befälhafvande amiral i Karlskrona. 1868 tog han afsked ur krigstjänsten."
test_string4 = "Ett sätt att mäta tid, består av två glaskupoler."
sentiment = predict_sentiment(test_string4, model, tokenizer, device)
print("Prediction: ", sentiment)

Prediction:  other
